In [1]:
import os, sys, ast
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from os import environ
from datetime import datetime, timedelta

In [2]:
output_dir = environ.get("output_dir",None)
dataset_filepath = environ.get("dataset_filepath",None)
input_dir = environ.get("input_dir",None)
words_dir = environ.get("words_dir",None)
results = environ.get("results", None)
stretch_image = ast.literal_eval(environ.get("stretch_image",None).title())
add_date = ast.literal_eval(environ.get("add_date",None).title())
skip_if_file_exists = ast.literal_eval(environ.get("skip_if_file_exists",None).title())
skip_labels = ast.literal_eval(environ.get("skip_labels","[]"))

AttributeError: 'NoneType' object has no attribute 'title'

In [3]:
def imhist(im):
    m, n = im.shape
    h = [0.0] * 256
    for i in range(m):
        for j in range(n):
            h[im[i, j]]+=1
    return np.array(h)/(m*n)

def cumsum(h):
    # finds cumulative sum of a numpy array, list
    return [sum(h[:i+1]) for i in range(len(h))]

def histeq(im):
    #calculate Histogram
    h = imhist(im)
    cdf = np.array(cumsum(h)) #cumulative distribution function
    sk = np.uint8(255 * cdf) #finding transfer function values
    s1, s2 = im.shape
    Y = np.zeros_like(im)
    # applying transfered values for each pixels
    for i in range(0, s1):
        for j in range(0, s2):
            Y[i, j] = sk[im[i, j]]
    H = imhist(Y)
    #return transformed image, original and new istogram, 
    # and transform function
    return Y , h, H, sk

def hist_stretching(im_asarray):
    mi = im_asarray.min()
    ma = im_asarray.max()
    gap = 255 / (ma - mi)
    
    Y = np.zeros_like(im_asarray)
    s1, s2 = im_asarray.shape

    for i in range(0, s1):
        for j in range(0, s2):
            Y[i, j] = (im_asarray[i, j] - mi) * gap
    
    return Y

In [4]:
'''
path: path of the image to draw on
output_path: path where to save the image to
score: value from 0 to 1 with the prediction result
label: prediction's name to print on image
stretch_image: Bool indicating if the image should be converted to it's stretched version
add_date: Bool indicating if date should be added to the frame.
'''
def draw_classification_on_image(path, output_path, score, label, stretch_image=False, add_date=True):
    # open image
    image = Image.open(path)

    if image.mode != "RGB":
        image=image.convert("RGB")
    
    # First of all, we stretch the image if needed
    if stretch_image:
        image = np.asarray(image) 

        r = image[:,:,0]
        eq_1 = hist_stretching(r)  #eq_1,_,_,_ = histeq(r)
        image = Image.fromarray(eq_1, 'L')
    
        if image.mode != "RGB":
            image=image.convert("RGB")
        
    # Once stretched, resize it to make it more 'uploadable'
    image = image.resize((600, 600), Image.ANTIALIAS)    
    
    # Now the frame's label
    draw = ImageDraw.Draw(image)
    
    # Once the image was sized up, now we write on top of it. First the date (if needed)
    if add_date:
        # name_of_video = "Users_thermaldata_unkown_2016-06-20_18%3A46%3A22.000000_2"
        name_of_video = os.path.basename(os.path.dirname(path))

        # str_date_of_video = 2016-06-20T18%3A46%3A22
        str_date_of_video = "T".join(os.path.splitext(name_of_video)[0].split("_")[-2:])

        # date_of_video = datetime(2016-06-20 18:46:22)
        date_of_video = datetime.strptime(str_date_of_video,"%Y-%m-%dT%H%%3A%M%%3A%S")

        #convert to argentinian date
        date_of_video -= timedelta(hours=3)

        font = ImageFont.truetype("SF-UI-Text-Medium.otf", 30)
        draw.text((20, image.size[0]* 3/4), 
                  text=date_of_video.strftime("%Y-%m-%d %H:%M"), 
                  fill="rgb(255,255,255)", font=font)

    # values for green box
    s = score*100
    width = 1.5*s
    max_width = 1.5*100
    
    # draw label name
    fontPath = "./SF-UI-Text-Medium.otf"
    fontSize = 40
    font  =  ImageFont.truetype ( fontPath, fontSize )
    x = 19
    y = 10
    
    '''
    # borde fino
    draw.text((x-1, y), text, font=font, fill="rgb(0,0,0)")
    draw.text((x+1, y), text, font=font, fill="rgb(0,0,0)")
    draw.text((x, y-1), text, font=font, fill="rgb(0,0,0)")
    draw.text((x, y+1), text, font=font, fill="rgb(0,0,0)")
    '''

    # borde grueso
    draw.text((x-1, y-1), text=label, font=font, fill="rgb(0,0,0)")
    draw.text((x+1, y-1), text=label, font=font, fill="rgb(0,0,0)")
    draw.text((x-1, y+1), text=label, font=font, fill="rgb(0,0,0)")
    draw.text((x+1, y+1), text=label, font=font, fill="rgb(0,0,0)")

    # draw label on image
    draw.text((x,y), text=label, fill="rgb(255,255,255)", font=font)

    #draw score label
    color_fill = "rgb(0,200,0)"
    draw.rectangle([(20,65),(20+max_width,70)], fill=None, outline=color_fill)
    draw.rectangle([(20,65),(20+width,70)], fill=color_fill, outline=color_fill)
    del draw

    # Save image to output_path
    image.save(output_path, format='PNG', quality=100)

In [ ]:
if not output_dir or not input_dir or not dataset_filepath or \
   not words_dir or not results or stretch_image is None or \
   add_date is None and skip_if_file_exists is None:
    raise "error"
else:
    with open(results, "r") as rf:
        results_lines = rf.readlines()

    with open(dataset_filepath, "r") as df:
        lines = df.readlines()
        dataset_files = {}
        for p in lines:
            if len(p.split()) == 2:
                dataset_files[p.split()[1]]=True
        
    # words_list = ["class1", "class2",..,"classn"]
    with open(words_dir, "r") as words:
        words_list = words.readlines()
        words_list = [item.split(' ')[0] for item in words_list]

    for result in results_lines:
        # name = 1 <path>
        p = result.split(' ')[0]
        if not dataset_files.get(p):
            continue
        
        video_name = p.split('/')[-2]
        frame_name = p.split('/')[-1][:-4]
        frame_number = int(frame_name.split('_')[0])*3 + int(frame_name.split('_')[1])
        p = os.path.dirname(os.path.dirname(p))
        p = os.path.join(p, video_name, frame_name + ".png")

        o = os.path.join(output_dir, p)
        p = os.path.join(input_dir, p)

        s = float(result.split(' ')[2])
        l = words_list[int(result.split(' ')[1])-1]

        if skip_if_file_exists and os.path.exists(o):
            continue
        
        if l in skip_labels:
            continue
            
        if not os.path.exists(os.path.dirname(o)):
            os.makedirs(os.path.dirname(o))
        draw_classification_on_image(p, o, s, l, stretch_image, add_date)
        print "✓ %s" %o

    print('finished printing on images')